In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [53]:
npf = pd.read_csv('npf_train.csv')
npf.index=npf['date']
npf['class4'] = npf['class4'].astype('category')
npf.drop(['id', 'partlybad', 'date'], axis=1, inplace=True)
npf.rename(columns={'class4':'class'}, inplace=True)
npf.head()

,class,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,...,T672.mean,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,nonevent,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,383.885077,1.955198,19.245511,...,-13.016471,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733
2000-01-20,nonevent,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,374.674177,0.435480,31.107659,...,-8.997430,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572
2000-01-23,nonevent,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,373.275062,0.165500,29.800885,...,-10.224472,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048
2000-02-17,nonevent,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,378.316909,1.983430,23.795211,...,-1.535183,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751
2000-03-25,Ib,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,...,-2.095641,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210


In [54]:
binary = npf.copy()
new_class = np.array(['event']*len(binary), dtype='object')
new_class[binary['class'] == 'nonevent'] = 'nonevent'
binary['class2'] = new_class
binary.drop('class', axis=1, inplace=True)
binary.rename(columns={'class2':'class'}, inplace=True)
binary.head()

,CO2168.mean,CO2168.std,CO2336.mean,CO2336.std,CO242.mean,CO242.std,CO2504.mean,CO2504.std,Glob.mean,Glob.std,...,T672.std,T84.mean,T84.std,UV_A.mean,UV_A.std,UV_B.mean,UV_B.std,CS.mean,CS.std,class
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,384.462000,2.284996,384.164462,2.135062,385.274688,2.211695,383.885077,1.955198,19.245511,11.909549,...,0.525698,-12.422972,0.376324,1.635563,0.856948,0.026438,0.014617,0.003374,0.000733,nonevent
2000-01-20,374.884615,0.415185,374.703333,0.385179,375.621266,0.665720,374.674177,0.435480,31.107659,24.624718,...,0.373927,-8.351043,0.575679,1.441109,0.741088,0.022649,0.012479,0.001501,0.000572,nonevent
2000-01-23,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,373.275062,0.165500,29.800885,22.892316,...,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048,nonevent
2000-02-17,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,378.316909,1.983430,23.795211,16.178905,...,0.122651,-0.829524,0.134191,2.261805,1.345651,0.030893,0.021903,0.002038,0.000751,nonevent
2000-03-25,373.128684,1.096617,372.980000,1.047750,373.701830,1.259198,372.910000,1.004164,252.480327,138.921953,...,1.695622,-1.095864,2.090111,12.906779,7.022300,0.333523,0.239981,0.000662,0.000210,event


In [4]:
#Multiclass Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = GaussianNB()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
multiclass_GNB = np.array(accuracies).mean()
print(multiclass_GNB)

0.5347342995169082


In [5]:
#Binary Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = GaussianNB()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
binary_GNB = np.array(accuracies).mean()
print(binary_GNB)

0.7646859903381642


In [6]:
#Multiclass Dummy Classifier
from sklearn.dummy import DummyClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = DummyClassifier(strategy='most_frequent')
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
multiclass_dummy = np.array(accuracies).mean()
print(multiclass_dummy)

0.5001932367149757


In [7]:
#Binary Dummy Classifier
from sklearn.dummy import DummyClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]
    
    model = DummyClassifier(strategy='most_frequent')
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))
binary_dummy = np.array(accuracies).mean()
print(binary_dummy)

0.4542512077294686


In [8]:
#Multiclass k-Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
max_neighbors = 20
multiclass_kNN_acc = list()
for i in range(max_neighbors):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = KNeighborsClassifier(n_neighbors=i+1)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    multiclass_kNN_acc.append(np.array(accuracies).mean())

multiclass_kNN = pd.DataFrame()
multiclass_kNN['Neighbors'] = range(1, max_neighbors+1)
multiclass_kNN['Accuracy'] = multiclass_kNN_acc
multiclass_kNN.index = multiclass_kNN['Neighbors']
multiclass_kNN.drop('Neighbors', axis=1, inplace=True)
multiclass_kNN = multiclass_kNN.sort_values(by='Accuracy', ascending=False)
multiclass_kNN

,Accuracy
Neighbors,
14,0.598599
9,0.591787
13,0.591691
5,0.587633
17,0.587585
6,0.587150
7,0.583188
11,0.582899
4,0.581014


In [9]:
#Binary k-Nearest Neighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
max_neighbors = 20
binary_kNN_acc = list()
for i in range(max_neighbors):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = KNeighborsClassifier(n_neighbors=i+1)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    binary_kNN_acc.append(np.array(accuracies).mean())

binary_kNN = pd.DataFrame()
binary_kNN['Neighbors'] = range(1, max_neighbors+1)
binary_kNN['Accuracy'] = binary_kNN_acc
binary_kNN.index = binary_kNN['Neighbors']
binary_kNN.drop('Neighbors', axis=1, inplace=True)
binary_kNN = binary_kNN.sort_values(by='Accuracy', ascending=False)
binary_kNN

,Accuracy
Neighbors,
8,0.801353
9,0.799469
3,0.792319
14,0.790531
19,0.790338
13,0.788551
18,0.788551
15,0.786232
17,0.786184


In [10]:
#Binary Perceptron
from sklearn.linear_model import Perceptron

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]

    model = Perceptron()
    model.fit(features_train, target_train)
    accuracies.append(model.score(features_test, target_test))

binary_perceptron = np.array(accuracies).mean()
print(binary_perceptron)

0.6753140096618357


In [11]:
#Chained Binary Perceptron
from sklearn.linear_model import Perceptron

k = 10
kf = KFold(n_splits=k, shuffle=True)
accuracies = list()

for train_index, test_index in kf.split(npf):
    train = npf.iloc[train_index]
    test = npf.iloc[test_index]
    
    EvNE_train = train.copy()
    temp = np.array(['event']*len(EvNE_train), dtype='object')
    temp[EvNE_train['class'] == 'nonevent'] = 'nonevent'
    EvNE_train['class'] = temp
    
    IvII_train = train.copy()
    IvII_train = IvII_train[IvII_train['class'] != 'nonevent']
    temp = np.array(['Type I']*len(IvII_train), dtype='object')
    temp[IvII_train['class'] == 'II'] = 'II'
    IvII_train['class'] = temp
    
    AvB_train = train.copy()
    AvB_train = AvB_train[(AvB_train['class'] == 'Ia') | (AvB_train['class'] == 'Ib')]
    temp = np.array(['Ia']*len(AvB_train), dtype='object')
    temp[AvB_train['class'] == 'Ib'] = 'Ib'
    AvB_train['class'] = temp
    
    EvNE_model = Perceptron()
    IvII_model = Perceptron()
    AvB_model = Perceptron()
    EvNE_model.fit(EvNE_train.drop('class', axis=1), EvNE_train['class'])
    IvII_model.fit(IvII_train.drop('class', axis=1), IvII_train['class'])
    AvB_model.fit(AvB_train.drop('class', axis=1), AvB_train['class'])
    
    predictions = list()
    features_test = test.drop('class', axis=1)
    target_test = test['class']
    for date in test.index:
        entry = features_test.loc[date]
        pred = EvNE_model.predict([entry]) #Compare Event vs Nonevent
        if pred == 'nonevent':
            predictions.append(pred[0])
        else:
            pred = IvII_model.predict([entry]) #Compare Type I vs Type II
            if pred == 'Type II':
                predictions.append(pred[0])
            else:
                pred = AvB_model.predict([entry]) #Compare Type Ia vs Type Ib
                predictions.append(pred[0])
    predictions = np.array(predictions)            
    correct_arr = (target_test == predictions).values
    correct = len(predictions[correct_arr])
    accuracies.append(correct/len(predictions))
    
chained_perceptron = np.array(accuracies).mean()
chained_perceptron

0.5415942028985508

In [12]:
#Multiclass Decision Tree
from sklearn.tree import  DecisionTreeClassifier

features = npf.drop('class', axis=1)
target = npf['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
split_acc = list()
for i in range(2, len(features)+1):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = DecisionTreeClassifier(min_samples_split=i)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    multiclass_tree = np.array(accuracies).mean()
    split_acc.append(multiclass_tree)
    
tree_table = pd.DataFrame()
tree_table['Split'] = list(range(2, len(features)+1))
tree_table['Accuracy'] = split_acc
tree_table.sort_values(by='Accuracy', ascending=False, inplace=True)
print(tree_table)

     Split  Accuracy
105    107  0.659324
150    152  0.657246
128    130  0.653140
151    153  0.652850
114    116  0.650821
..     ...       ...
426    428  0.499807
454    456  0.499807
419    421  0.499758
433    435  0.499662
418    420  0.499662

[457 rows x 2 columns]


In [13]:
#Binary Decision Tree
from sklearn.tree import  DecisionTreeClassifier

features = binary.drop('class', axis=1)
target = binary['class']

k = 10
kf = KFold(n_splits=k, shuffle=True)
split_acc = list()
for i in range(2, len(features)+1):
    accuracies = list()
    for train_index, test_index in kf.split(features):
        features_train = features.iloc[train_index]
        features_test = features.iloc[test_index]
        target_train = target.iloc[train_index]
        target_test = target.iloc[test_index]

        model = DecisionTreeClassifier(min_samples_split=i)
        model.fit(features_train, target_train)
        accuracies.append(model.score(features_test, target_test))
    binary_tree = np.array(accuracies).mean()
    split_acc.append(binary_tree)
    
binary_tree_table = pd.DataFrame()
binary_tree_table['Split'] = list(range(2, len(features)+1))
binary_tree_table['Accuracy'] = split_acc
binary_tree_table.sort_values(by='Accuracy', ascending=False, inplace=True)
print(binary_tree_table)

     Split  Accuracy
206    208  0.838309
207    209  0.838261
204    206  0.838261
200    202  0.836184
203    205  0.835797
..     ...       ...
418    420  0.430000
435    437  0.425942
448    450  0.425894
425    427  0.425700
453    455  0.410290

[457 rows x 2 columns]


In [16]:
#Binary Random Forest
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(n_jobs=-1)

features = binary.drop('class', axis=1)
target = binary['class']

param_grid = {
    'n_estimators':list(range(50, 1001, 50)),                       
    'max_features': ['auto','sqrt'], 
    'max_depth': list(range(50, 1001, 50))+[None],
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11)),
    'bootstrap': [True,False]
}

grid = RandomizedSearchCV(RFC, cv=10, param_distributions=param_grid, n_iter=100)
grid.fit(features, target)

print(grid.best_estimator_)

df=pd.DataFrame(grid.cv_results_)
df.sort_values(by=['rank_test_score', 'mean_test_score', 'std_test_score'], ascending=(True, False, True), inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

print(df.loc[0, 'mean_test_score'])
binary_random_forest_table = df.copy()
binary_forest = binary_random_forest_table.loc[0, 'mean_test_score']

RandomForestClassifier(max_depth=700, max_features='sqrt', min_samples_leaf=8,
                       min_samples_split=6, n_estimators=150, n_jobs=-1)
0.871207729468599


In [17]:
#Binary Random Forest
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(n_jobs=-1)

features = npf.drop('class', axis=1)
target = npf['class']

param_grid = {
    'n_estimators':list(range(50, 1001, 50)),                       
    'max_features': ['auto','sqrt'], 
    'max_depth': list(range(50, 1001, 50))+[None],
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11)),
    'bootstrap': [True,False]
}

grid = RandomizedSearchCV(RFC, cv=10, param_distributions=param_grid, n_iter=100)
grid.fit(features, target)

print(grid.best_estimator_)

df=pd.DataFrame(grid.cv_results_)
df.sort_values(by=['rank_test_score', 'mean_test_score', 'std_test_score'], ascending=(True, False, True), inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

print(df.loc[0, 'mean_test_score'])
multiclass_random_forest_table = df.copy()
multiclass_forest = multiclass_random_forest_table.loc[0, 'mean_test_score']

RandomForestClassifier(bootstrap=False, max_depth=950, min_samples_leaf=10,
                       min_samples_split=3, n_estimators=550, n_jobs=-1)
0.6615458937198068


In [55]:
binary_arr = [binary_dummy, binary_forest, binary_GNB, binary_kNN_acc[0], binary_perceptron, binary_tree]
multiclass_arr = [multiclass_dummy, multiclass_forest, multiclass_GNB, multiclass_kNN_acc[0], chained_perceptron, 
                  multiclass_tree]
names = ['Dummy', 'Random Forest', 'Gaussian Naive Bayes', 'k-Nearest Neighbors', 'Perceptron', 'Binary Tree']
accuracy_table = pd.DataFrame(data=[binary_arr, multiclass_arr], columns=names)
accuracy_table.index = ['Binary', 'Multiclass']
accuracy_table.index.name = 'Category'
accuracy_table = accuracy_table.transpose()
accuracy_table.head()

Category,Binary,Multiclass
Dummy,0.454251,0.500193
Random Forest,0.871208,0.661546
Gaussian Naive Bayes,0.764686,0.534734
k-Nearest Neighbors,0.772705,0.539469
Perceptron,0.675314,0.541594


In [89]:
forest_features = pd.read_csv('rf_binary.csv')
forest_features.drop('Unnamed: 0', axis=1, inplace=True)
forest_features.sort_values(by='Overall', ascending=False, inplace=True)

binary_n = 5 #number of features to take
target = binary['class']
binary_feature_names = forest_features.loc[:binary_n-1, 'names'].to_numpy() 
binary_features = binary[binary_feature_names]

param_grid = {
    'n_estimators':list(range(50, 1001, 50)),                       
    'max_features': ['auto','sqrt'], 
    'max_depth': list(range(50, 1001, 50))+[None],
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11)),
    'bootstrap': [True,False]
}

grid = RandomizedSearchCV(RFC, cv=10, param_distributions=param_grid, n_iter=100)
grid.fit(features, target)

print(grid.best_estimator_)

binary_random_forest_final_table=pd.DataFrame(grid.cv_results_)
binary_random_forest_final_table.sort_values(by=['rank_test_score', 'mean_test_score', 'std_test_score'], 
                                       ascending=(True, False, True), inplace=True)
binary_random_forest_final_table.reset_index(drop=True, inplace=True)
binary_random_forest_final_table.head()

binary_random_forest_final = binary_random_forest_final_table.loc[0]

RandomForestClassifier(bootstrap=False, max_depth=600, min_samples_split=10,
                       n_estimators=900, n_jobs=-1)


In [100]:
forest_features = pd.read_csv('rf_full.csv')
forest_features.drop('Unnamed: 0', axis=1, inplace=True)
forest_features.sort_values(by='Overall', ascending=False, inplace=True)

multiclass_n = 5 #number of features to take
target = npf['class']
multiclass_feature_names = forest_features.loc[:multiclass_n-1, 'names'].to_numpy() 
multiclass_features = npf[multiclass_feature_names]

param_grid = {
    'n_estimators':list(range(50, 1001, 50)),                       
    'max_features': ['auto','sqrt'], 
    'max_depth': list(range(50, 1001, 50))+[None],
    'min_samples_split': list(range(2, 11)),
    'min_samples_leaf': list(range(1, 11)),
    'bootstrap': [True,False]
}

grid = RandomizedSearchCV(RFC, cv=10, param_distributions=param_grid, n_iter=100)
grid.fit(features, target)

print(grid.best_estimator_)

multiclass_random_forest_final_table=pd.DataFrame(grid.cv_results_)
multiclass_random_forest_final_table.sort_values(by=['rank_test_score', 'mean_test_score', 'std_test_score'], 
                                       ascending=(True, False, True), inplace=True)
multiclass_random_forest_final_table.reset_index(drop=True, inplace=True)
multiclass_random_forest_final_table.head()

multiclass_random_forest_final = binary_random_forest_final_table.loc[0]

RandomForestClassifier(bootstrap=False, max_depth=900, max_features='sqrt',
                       min_samples_leaf=7, min_samples_split=5,
                       n_estimators=750, n_jobs=-1)


In [101]:
binary_random_forest_final = binary_random_forest_final[['param_n_estimators', 'param_min_samples_split', 
                                                                 'param_min_samples_leaf', 'param_max_features', 
                                                                 'param_max_depth', 'param_bootstrap']]
multiclass_random_forest_final = multiclass_random_forest_final[['param_n_estimators', 'param_min_samples_split', 
                                                                 'param_min_samples_leaf', 'param_max_features', 
                                                                 'param_max_depth', 'param_bootstrap']]

In [102]:
binary_random_forest_final

param_n_estimators           900
param_min_samples_split       10
param_min_samples_leaf         1
param_max_features          auto
param_max_depth              600
param_bootstrap            False
Name: 0, dtype: object

In [122]:
#Binary Random Forest - Test Data

test_data = pd.read_csv('npf_test_hidden.csv')
multiclass_test_features = test_data[multiclass_feature_names]

params = multiclass_random_forest_final
multiclass_RCF = RandomForestClassifier(n_jobs=-1, n_estimators = params[0], min_samples_split=params[1], 
                                    min_samples_leaf=params[2], max_features=params[3], max_depth=params[4], 
                                    bootstrap=params[5])
multiclass_RCF.fit(multiclass_features, npf['class'])
multiclass_output = pd.DataFrame()
multiclass_output['class4'] = multiclass_RCF.predict(multiclass_test_features)
multiclass_output['nonevent'] = np.array(multiclass_RCF.predict_proba(multiclass_test_features))[:, 3]
multiclass_output['p'] = np.ones(len(multiclass_output)) - np.array(multiclass_output['nonevent'])
multiclass_output.drop('nonevent', axis=1, inplace=True)

In [141]:
train_predict = multiclass_RCF.predict(npf[multiclass_feature_names])
train_targets = npf['class']
train_check = pd.DataFrame()
train_check.index = npf.index
train_check['Prediction'] = train_predict
train_check['True'] = train_targets

correct = 0
for date in train_check.index:
    prediction = train_check.loc[date, 'Prediction']
    true = train_check.loc[date, 'True']
    if (prediction == 'nonevent' and true == 'nonevent') or (prediction != 'nonevent' and true != 'nonevent'):
        correct += 1
        
predicted_binary_accuracy = correct/len(npf)
predicted_binary_accuracy

0.9912663755458515

In [142]:
len(train_check[train_check['Prediction'] == train_check['True']])

453

In [162]:
accuracies = list()
features = npf[multiclass_feature_names]
target = npf['class']

binary_acc = list()
for train_index, test_index in kf.split(features):
    features_train = features.iloc[train_index]
    features_test = features.iloc[test_index]
    target_train = target.iloc[train_index]
    target_test = target.iloc[test_index]

    model = RandomForestClassifier(n_jobs=-1, n_estimators = params[0], min_samples_split=params[1], min_samples_leaf=params[2],
                                   max_features=params[3], max_depth=params[4], bootstrap=params[5])
    model.fit(features_train, target_train)
    
    train_predict = model.predict(features_test[multiclass_feature_names])
    train_check = pd.DataFrame()
    train_check['Prediction'] = train_predict
    train_check['True'] = target_test.values

    correct = 0
    for date in train_check.index:
        prediction = train_check.loc[date, 'Prediction']
        true = train_check.loc[date, 'True']
        if (prediction == 'nonevent' and true == 'nonevent') or (prediction != 'nonevent' and true != 'nonevent'):
            correct += 1

    binary_acc.append(correct/len(train_check))
    
binary_acc = np.array(binary_acc)
predicted_binary_accuracy = binary_acc.mean()

In [163]:
predicted_binary_accuracy

0.8255072463768117

In [175]:
output1 = [round(predicted_binary_accuracy, 2), 'class4'] + list(multiclass_output['class4'].values)
output2 = ['', 'p'] + list(multiclass_output['p'].values)
output = pd.DataFrame()
output['1'] = output1
output['2'] = output2
output.to_csv('prediction.csv', index=False, header=False)